In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# xây dựng graph (mạng liên kết)
# từ mạng liên kết -> embedding
# embedding gán nhãn (tiêu chí đánh giá) (trung bình điểm học sinh trong khóa học đó)
# train model

In [ ]:
!pip install node2vec

In [ ]:
!git clone https://github.com/THU-KEG/MOOCCubeX.git

Cloning into 'MOOCCubeX'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 61 (delta 4), reused 0 (delta 0), pack-reused 53 (from 1)
Receiving objects: 100% (61/61), 1.29 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
root_folder = "/content/mooc/raw"

In [ ]:
!wget http://lfs.aminer.cn/misc/moocdata/data/MOOCCube.zip
!unzip MOOCCube.zip -d /content/mooc/

--2024-12-11 11:56:09--  http://lfs.aminer.cn/misc/moocdata/data/MOOCCube.zip
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:80... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 1090812978 (1.0G) [application/zip]
Saving to: ‘MOOCCube.zip’

MOOCCube.zip        100%[===================>]   1.02G  7.76MB/s    in 2m 19s  

2024-12-11 11:58:29 (7.47 MB/s) - ‘MOOCCube.zip’ saved [1090812978/1090812978]

Archive:  MOOCCube.zip
   creating: /content/mooc/MOOCCube/
   creating: /content/mooc/MOOCCube/entities/
  inflating: /content/mooc/MOOCCube/entities/course.json  
  inflating: /content/mooc/MOOCCube/entities/paper.json  
  inflating: /content/mooc/MOOCCube/entities/school.json  
  inflating: /content/mooc/MOOCCube/entities/user.json  
  inflating: /content/mooc/MOOCCube/entities/video.json  
  inflating: /content/mooc/MOOCCube/entities/teacher.json  
  inflat

In [ ]:
import json
import os
import pandas as pd
import csv
import io

def read_json_relation(file_path, save_path, node_title):
  with open(file_path, 'r', encoding='utf-8') as infile, open(save_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)

    writer.writerow(node_title)
    for index, line in enumerate(infile):
        try:
          data = line.strip().split('\t')
          writer.writerow(data)
        except:
          print(data)
  return pd.read_csv(save_path)

def get_data_csv(file_path, save_path, limit = -1):
  with open(file_path, 'r', encoding='utf-8') as infile, open(save_path, 'w', newline='', encoding='utf-8') as outfile:

    writer = csv.writer(outfile)
    for index, line in enumerate(infile):
        try:
          data = json.loads(line)
          if index == 0:
            writer.writerow(data.keys())
          writer.writerow(data.values())
          if index + 1 == limit:
            break
        except:
          print(data)
  return pd.read_csv(save_path)


In [ ]:
user = get_data_csv("/content/mooc/MOOCCube/entities/user.json", "/content/drive/MyDrive/mooc/csv/user.csv")

print(user.head())

           id name                                       course_order  \
0   U_7001215  李喜锋  ['C_course-v1:TsinghuaX+00740043_2x_2015_T2+sp...   
1  U_10402446  五元香  ['C_course-v1:TsinghuaX+00510888X+2019_T1', 'C...   
2  U_10359065   魏珊  ['C_course-v1:TsinghuaX+00612642X+sp', 'C_cour...   
3   U_7423998  郭海滨  ['C_course-v1:TsinghuaX+30240184_2X+sp', 'C_co...   
4    U_545306  李其艳  ['C_course-v1:TsinghuaX+20430064_2X+sp', 'C_co...   

                                         enroll_time  
0  ['2017-05-01 11:07:53', '2017-05-17 10:07:17',...  
1     ['2019-06-14 08:50:04', '2019-01-04 20:36:07']  
2     ['2019-01-18 21:19:56', '2019-01-14 21:54:54']  
3  ['2017-08-16 10:38:11', '2018-07-01 18:24:24',...  
4  ['2018-09-05 15:40:40', '2019-02-28 10:08:49',...  


In [ ]:
course = get_data_csv("/content/mooc/MOOCCube/entities/course.json", "/content/drive/MyDrive/mooc/csv/course.csv")

print(course.head())

                                              id                   name  \
0           C_course-v1:McGillX+ATOC185x+2015_T1             自然灾害（自主模式）   
1      C_course-v1:TsinghuaX+THESIS2015X+2015_T1  2015年清华大学研究生学位论文答辩（二）   
2  C_course-v1:TsinghuaX+THESIS2014_1X_tv+_2014_  2014年清华大学研究生学位论文答辩（一）   
3   C_course-v1:TsinghuaX+THESIS2015X_tv+2015_T1  2015年清华大学研究生学位论文答辩（一）   
4              C_course-v1:TsinghuaX+00690242+sp        文物精品与文化中国（自主模式）   

  prerequisites                                              about  \
0             无  <p>地球上没有一处地方不发生自然灾害。当我们以科学的眼光看待这些自然灾害的原因和本质时，我...   
1         无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
2         无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
3         无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
4             无  <p>中国考古学是以往100年中发展最为迅速的领域之一，大批珍贵文物的出土，不断刷新人们对文...   

                                         core_id  \
0           C_course-v1:McGillX+ATOC185x+2015_T1   
1      C_course-v1:Tsing

In [ ]:
school = get_data_csv("/content/mooc/MOOCCube/entities/school.json", "/content/drive/MyDrive/mooc/csv/school.csv")

print(school.head())

        id    name                                              about
0    S_BNU  北京师范大学  北京师范大学（Beijing Normal University）简称“北师大”，由中华人民...
1    S_UQx   昆士兰大学  昆士兰大学（The University of Queensland），简称昆大，世界50强...
2  S_UESTC  电子科技大学  电子科技大学（University of Electronic Science and Te...
3    S_BIT  北京理工大学  北京理工大学（Beijing Institute of Technology）简称北理工，由...
4    S_JNU    暨南大学  暨南大学（Jinan University），简称“”暨大（JNU），位于广东省广州市，是中...


In [ ]:
teacher = get_data_csv("/content/mooc/MOOCCube/entities/teacher.json", "/content/drive/MyDrive/mooc/csv/teacher.csv")

print(teacher.head())

      id name                                              about
0  T_方维奇  方维奇  方维奇，男，陕西工业职业技术学院教师，讲师。主持并参与院级科研项目3项，发表教科研论文13篇...
1  T_范茂魁  范茂魁  范茂魁（1979-）男，教授，硕士学位，教龄17年，主要从事《消防燃烧学》、《化学基础》、《...
2  T_连小珉  连小珉  连小珉，男，1955年生于重庆。清华大学教授，清华大学机械工程学位分委会副主席。中国汽车工程...
3  T_张德顺  张德顺  张德顺,1964年生,同济大学建筑与城市规划学院高密度人居环境生态与节能 教育部重点实验室教...
4   T_王红   王红  王红，硕士，教授，硕士生导师。长期从事寄生虫学教学 与科研工作。教学方面，独立系统地担任了本...


In [ ]:
file_url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json"
file_destination = os.path.join("/content", "course.json")

!wget -O "{file_destination}" "{file_url}"

--2024-12-09 11:01:38--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 44660123 (43M) [application/json]
Saving to: ‘/content/course.json’

/content/course.jso 100%[===================>]  42.59M  12.3MB/s    in 4.0s    

2024-12-09 11:01:43 (10.7 MB/s) - ‘/content/course.json’ saved [44660123/44660123]



In [ ]:
file_url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/problem.json"
file_destination = os.path.join("/content", "problem.json")

!wget -O "{file_destination}" "{file_url}"

--2024-12-08 08:21:47--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/problem.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 1290278596 (1.2G) [application/json]
Saving to: ‘/content/problem.json’

/content/problem.js 100%[===================>]   1.20G  13.6MB/s    in 91s     

2024-12-08 08:23:19 (13.5 MB/s) - ‘/content/problem.json’ saved [1290278596/1290278596]



In [ ]:
file_url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/exercise-problem.txt"
file_destination = os.path.join("/content", "exercise-problem.txt")

!wget -O "{file_destination}" "{file_url}"

--2024-12-08 15:42:59--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/exercise-problem.txt
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 135124979 (129M) [text/plain]
Saving to: ‘/content/exercise-problem.txt’

/content/exercise-p 100%[===================>] 128.86M  14.7MB/s    in 9.9s    

2024-12-08 15:43:10 (13.1 MB/s) - ‘/content/exercise-problem.txt’ saved [135124979/135124979]



In [ ]:
# các bài kiểm tra được bao câu hỏi
exercise_problem = read_json_relation("/content/exercise-problem.txt", "/content/drive/MyDrive/mooc/csv/exercise-problem.csv", ["excercise","problem"])

print(exercise_problem.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/exercise-problem.txt'

In [ ]:
user_course = read_json_relation("/content/mooc/MOOCCube/relations/user-course.json", "/content/drive/MyDrive/mooc/csv/user-course.csv", ["user","course"])

print(user_course.head())

        user                                        course
0  U_7001215  C_course-v1:TsinghuaX+00740043_2x_2015_T2+sp
1  U_7001215             C_course-v1:TsinghuaX+30240184+sp
2  U_7001215    C_course-v1:TsinghuaX+00740043X_2015_T2+sp
3  U_7001215     C_course-v1:TsinghuaX+10421094X_2015_2+sp
4  U_7001215          C_course-v1:TsinghuaX+30240184_2X+sp


In [ ]:
teacher_course = read_json_relation("/content/mooc/MOOCCube/relations/teacher-course.json", "/content/drive/MyDrive/mooc/csv/teacher-course.csv", ["teacher","course"])

print(teacher_course.head())

  teacher                                     course
0   T_方维奇            C_course-v1:SPI+20170828001x+sp
1   T_方维奇      C_course-v1:SXPI+20170828001x+2019_T1
2   T_范茂魁      C_course-v1:PSFFC+2018102405X+2018_T2
3   T_连小珉  C_course-v1:TsinghuaX+70150104_2X+2019_T1
4   T_连小珉    C_course-v1:TsinghuaX+70150104X+2019_T1


In [ ]:
school_teacher = read_json_relation("/content/mooc/MOOCCube/relations/school-teacher.json", "/content/drive/MyDrive/mooc/csv/school-teacher.csv", ["school","teacher"])

print(school_teacher.head())

  school teacher
0  S_BNU    T_张燕
1  S_BNU   T_陈蒨蒨
2  S_BNU   T_王文周
3  S_BNU   T_马俊伟
4  S_BNU   T_成卫东


In [ ]:
file_url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/user-problem.json"
file_destination = os.path.join("/content", "user-problem.json")

!wget -O "{file_destination}" "{file_url}"

--2024-12-08 14:27:15--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/user-problem.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 22453311654 (21G) [application/json]
Saving to: ‘/content/user-problem.json’

/content/user-probl 100%[===================>]  20.91G  12.4MB/s    in 28m 50s 

2024-12-08 14:56:06 (12.4 MB/s) - ‘/content/user-problem.json’ saved [22453311654/22453311654]



In [ ]:
import json
import os
import pandas as pd
import csv
import io

def read_json_relation(file_path, save_path, node_title):
  with open(file_path, 'r', encoding='utf-8') as infile, open(save_path, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)

    writer.writerow(node_title)
    for index, line in enumerate(infile):
        try:
          data = line.strip().split('\n')
          writer.writerow(data)
        except:
          print(data)
  return pd.read_csv(save_path)

def get_data_csv(file_path, save_path, limit = -1):
  with open(file_path, 'r', encoding='utf-8') as infile, open(save_path, 'w', newline='', encoding='utf-8') as outfile:

    writer = csv.writer(outfile)
    for index, line in enumerate(infile):
        try:
          data = json.loads(line)
          if index == 0:
            writer.writerow(data.keys())
          writer.writerow(data.values())
          if index + 1 == limit:
            break
        except:
          print(data)
  return pd.read_csv(save_path)


# user đã làm bài kiểm tra và được bao nhiêu điểm trên bài kiểm tra đó
# nhiều hàng mỗi hàng là 1 json
def user_problem(file_path, save_path):
  with open(file_path, 'r', encoding='utf-8') as infile, open(save_path, 'w', newline='', encoding='utf-8') as outfile:

    writer = csv.writer(outfile)
    writer.writerow(["problem_id","score"])
    for index, line in enumerate(infile):
        try:
          data = json.loads(line)
          score = data['score']
          problem_id = data['problem_id']
          if score != None:
            writer.writerow([problem_id, 0 if score < 0 else score])
        except:
          print(data)

In [ ]:
user_problem("/content/user-problem.json", "/content/drive/MyDrive/mooc/csv/user-problem.csv")

NameError: name 'user_problem' is not defined

In [ ]:
problem = pd.read_csv("/content/drive/MyDrive/mooc/csv/user-problem.csv")

In [ ]:
problem.head()

,problem_id,score
0,Pm_3624759,0.0
1,Pm_3624760,0.0
2,Pm_3624762,0.0
3,Pm_3640328,0.0
4,Pm_3720166,0.0


In [ ]:
exercise_df = pd.read_csv('/content/drive/MyDrive/mooc/csv/exercise-problem.csv')
problem_df = pd.read_csv('/content/drive/MyDrive/mooc/csv/user-problem.csv')

merged_df = pd.merge(exercise_df, problem_df, left_on='problem', right_on='problem_id', how='left')

exercise_score_df = merged_df.groupby('excercise')['score'].mean().reset_index() # Avg score

exercise_score_df.rename(columns={'score': 'average_score'}, inplace=True)

exercise_score_df.to_csv('/content/drive/MyDrive/mooc/csv/exercise_score.csv', index=False)

print(exercise_score_df)


        excercise  average_score
0       Ex_100000            NaN
1       Ex_100007            NaN
2       Ex_100009            NaN
3       Ex_100013            NaN
4       Ex_100018            NaN
...           ...            ...
888737   Ex_99971            NaN
888738   Ex_99981            NaN
888739   Ex_99988            NaN
888740   Ex_99994            NaN
888741   Ex_99996            NaN

[888742 rows x 2 columns]


In [ ]:
total_rows, total_columns = exercise_score_df.shape
print(f"Tổng số hàng: {total_rows}, Tổng số cột: {total_columns}")

nan_counts = exercise_score_df.isna().sum()
print("Số lượng NaN trong mỗi cột:")
print(nan_counts)

total_nan = nan_counts.sum()
print(f"Tổng số NaN trong toàn bộ dữ liệu: {total_nan}")

Tổng số hàng: 888742, Tổng số cột: 2
Số lượng NaN trong mỗi cột:
excercise             0
average_score    788537
dtype: int64
Tổng số NaN trong toàn bộ dữ liệu: 788537


In [ ]:
import networkx as nx

G = nx.DiGraph()

In [ ]:
for index, row in user.iterrows():
  G.add_node(row['id'], type='user')

In [ ]:
for index, row in school.iterrows():
  G.add_node(row['id'], type='school')

In [ ]:
for index, row in teacher.iterrows():
  G.add_node(row['id'], type='teacher')

In [ ]:
for index, row in course.iterrows():
  G.add_node(row['id'], type='course')

In [ ]:
for index, row in teacher_course.iterrows():
  G.add_edge(row['teacher'], row['course'], relationship='teacher_course')

In [ ]:
for index, row in user_course.iterrows():
  G.add_edge(row['user'], row['course'], relationship='user_course')

In [ ]:
for index, row in school_teacher.iterrows():
  G.add_edge(row['school'], row['teacher'], relationship='school_teacher')

In [ ]:
# Đếm số lượng node và cạnh
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print(f"Số lượng node: {num_nodes}")
print(f"Số lượng cạnh: {num_edges}")

Số lượng node: 201857
Số lượng cạnh: 687227


In [ ]:
data = nx.node_link_data(G)
with open("/content/drive/MyDrive/mooc/graph.json", "w") as f:
    json.dump(data, f)

/usr/local/lib/python3.10/dist-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


In [ ]:
from node2vec import Node2Vec
import numpy as np

node2vec = Node2Vec(G, dimensions=32, walk_length=10, num_walks=100, workers=4)

model = node2vec.fit()

Computing transition probabilities:   0%|          | 0/201857 [00:00<?, ?it/s]

In [ ]:
node_embeddings = {node: model.wv[node] for node in G.nodes()}

print("Embedding của Math101:", node_embeddings['C_course-v1:McGillX+ATOC185x+2015_T1'])

Embedding của Math101: [-1.9673861   1.3300624  -2.8454282  -0.7733412  -4.2136903  -2.594082
  2.7420573   0.04518626 -4.1995053   1.6061578  -2.5413144  -0.5918913
  0.38685268 -1.1992847   2.0592985   2.5611053  -0.2405438   2.1300774
  0.9715804   2.495877    1.6109236   2.4496546   0.42610264  1.6635199
  1.9702959   1.5906929  -0.24332961 -1.0886192  -1.1537926  -3.3355339
  0.515576    2.0850284 ]


In [ ]:
import pickle

with open('/content/drive/MyDrive/mooc/node_embeddings.pkl', 'wb') as f:
    pickle.dump(node_embeddings, f)

In [ ]:
file_url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json"
file_destination = os.path.join("/content", "course.json")

!wget -O "{file_destination}" "{file_url}"

course_v1 = pd.read_csv('/content/drive/MyDrive/mooc/csv/course.csv')

print(course_v1.head())

course_v2 = get_data_csv("/content/course.json", "/content/course.csv")

print(course_v2.head())



--2024-12-11 12:02:28--  https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json
Resolving lfs.aminer.cn (lfs.aminer.cn)... 159.27.2.14
Connecting to lfs.aminer.cn (lfs.aminer.cn)|159.27.2.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Could not parse String-Transport-Security header
Length: 44660123 (43M) [application/json]
Saving to: ‘/content/course.json’

/content/course.jso 100%[===================>]  42.59M  8.95MB/s    in 5.7s    

2024-12-11 12:02:35 (7.52 MB/s) - ‘/content/course.json’ saved [44660123/44660123]

                                              id                   name  \
0           C_course-v1:McGillX+ATOC185x+2015_T1             自然灾害（自主模式）   
1      C_course-v1:TsinghuaX+THESIS2015X+2015_T1  2015年清华大学研究生学位论文答辩（二）   
2  C_course-v1:TsinghuaX+THESIS2014_1X_tv+_2014_  2014年清华大学研究生学位论文答辩（一）   
3   C_course-v1:TsinghuaX+THESIS2015X_tv+2015_T1  2015年清华大学研究生学位论文答辩（一）   
4              C_course-v1:TsinghuaX+00690242+sp        文物精品

In [ ]:

file1 = course_v1
file2 = course_v2

file1['name'] = file1['name'].str.strip().str.lower()
file2['name'] = file2['name'].str.strip().str.lower()

merged_df = pd.merge(file1, file2[['name', 'resource']], on='name', how='left')
merged_df = merged_df.dropna(subset=['resource'])
print(merged_df)

                                                id                   name  \
1        C_course-v1:TsinghuaX+THESIS2015X+2015_T1  2015年清华大学研究生学位论文答辩（二）   
2    C_course-v1:TsinghuaX+THESIS2014_1X_tv+_2014_  2014年清华大学研究生学位论文答辩（一）   
3     C_course-v1:TsinghuaX+THESIS2015X_tv+2015_T1  2015年清华大学研究生学位论文答辩（一）   
10          C_course-v1:TsinghuaX+JWWCD001+2015_T2                 教＆学微辞典   
60    C_course-v1:TsinghuaX+THESIS2016X_tv+2016_T1  2016年清华大学研究生学位论文答辩（二）   
..                                             ...                    ...   
736            C_course-v1:NDU+2019042601X+2019_T2                   军事理论   
737            C_course-v1:CSU+2019043001X+2019_T1                医患沟通技能学   
739             C_course-v1:SYJU+030020406+2019_T1            暖通空调(2019春)   
740           C_course-v1:HBNU+2019051509X+2019_T1                5g与人工智能   
741           C_course-v1:HBNU+2019051509X+2019_T1                5g与人工智能   

                         prerequisites  \
1                                

In [ ]:
import re

extracted_exercise = []

for index, row in merged_df.iterrows():
    json_string = row['resource']

    regex_pattern = r"'resource_id'\s*:\s*'Ex_\w+'"

    matches = re.findall(regex_pattern, json_string)

    resource_ids = [match.split(":")[-1].strip().strip("'") for match in matches]

    extracted_exercise.append(resource_ids)

merged_df['exercise'] = extracted_exercise

print(merged_df.head())

                                               id                   name  \
1       C_course-v1:TsinghuaX+THESIS2015X+2015_T1  2015年清华大学研究生学位论文答辩（二）   
2   C_course-v1:TsinghuaX+THESIS2014_1X_tv+_2014_  2014年清华大学研究生学位论文答辩（一）   
3    C_course-v1:TsinghuaX+THESIS2015X_tv+2015_T1  2015年清华大学研究生学位论文答辩（一）   
10         C_course-v1:TsinghuaX+JWWCD001+2015_T2                 教＆学微辞典   
60   C_course-v1:TsinghuaX+THESIS2016X_tv+2016_T1  2016年清华大学研究生学位论文答辩（二）   

     prerequisites                                              about  \
1            无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
2            无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
3            无先修要求  <p>学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建学术交流的平...   
10  本课程为基础课，无先修要求。  <p>2014年10月17日，清华大学发布了《清华大学关于全面深化教育教学改革的若干意见》（...   
60              --  <p>\r\n\t学位论文答辩环节是研究生培养的重要环节，为了充分发挥该环节的育人作用，搭建...   

                                          core_id  \
1       C_course-v1:TsinghuaX+THESIS2015X+2015_T1  

In [ ]:
exercise_score = pd.read_csv('/content/drive/MyDrive/mooc/csv/exercise_score.csv')

exercise_score.head()

,excercise,average_score
0,Ex_100000,NaN
1,Ex_100007,NaN
2,Ex_100009,NaN
3,Ex_100013,NaN
4,Ex_100018,NaN


In [ ]:
import pandas as pd

course_data = []

for index, row in merged_df.iterrows():
    course_id = row['id']
    exercise_ids = row['exercise']

    scores = exercise_score[exercise_score['excercise'].isin(exercise_ids)]['average_score']

    avg_score = scores.mean() if not scores.empty else None

    if avg_score is None:
        label = 1
    elif 0 < avg_score < 50: # yếu
        label = 0
    elif 50 <= avg_score < 80: # tốt
        label = 1
    elif 80 <= avg_score <= 100: # giỏi
        label = 2
    else:
        label = 0

    course_data.append({'id': course_id, 'label': label})

course_df = pd.DataFrame(course_data)

course_df.to_csv("course_score.csv", index=False)

In [ ]:
ids = course_df['id'].tolist()
labels = course_df['label'].tolist()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
data = []
for node in G.nodes():
    for index, id in enumerate(ids):
      if node == id:
        embedding = node_embeddings[node]
        quality = labels[index]
        data.append(list(embedding) + [quality])

In [ ]:
df = pd.DataFrame(data)

X = df.iloc[:, :-1].values # 32x1
y = df.iloc[:, -1].values # number 3


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        29
           1       0.25      1.00      0.40         1

    accuracy                           0.90        30
   macro avg       0.62      0.95      0.67        30
weighted avg       0.97      0.90      0.93        30



In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

y_pred_rf = model_rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        29
           1       0.33      1.00      0.50         1

    accuracy                           0.93        30
   macro avg       0.67      0.97      0.73        30
weighted avg       0.98      0.93      0.95        30



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_gbm = GradientBoostingClassifier(n_estimators=100, random_state=42)
model_gbm.fit(X_train, y_train)

y_pred_rf = model_gbm.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.86      0.93        29
           1       0.20      1.00      0.33         1

    accuracy                           0.87        30
   macro avg       0.60      0.93      0.63        30
weighted avg       0.97      0.87      0.91        30

